In [48]:
import os
import pandas as pd
import json
import traceback
from dotenv import load_dotenv
from langchain_huggingface import HuggingFacePipeline
from langchain import PromptTemplate, HuggingFaceHub, LLMChain



In [51]:
from langchain.chat_models import ChatOpenAI

load_dotenv()

True

In [52]:
KEY=os.getenv("OPENAI_API_KEY")
KEY2=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [53]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [54]:
llm_hf = HuggingFaceHub(
    repo_id='google/flan-t5-large',  # The Hugging Face model repo ID
    model_kwargs={'temperature': 1.5}  # Set your model kwargs (e.g., temperature)
)

c:\Users\Administrator\Documents\mcqgen\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [55]:
llm_hf

HuggingFaceHub(client=<InferenceClient(model='google/flan-t5-large', timeout=None)>, repo_id='google/flan-t5-large', task='text2text-generation', model_kwargs={'temperature': 1.5})

In [18]:
# from langchain.llms import OpenAI
# from langchain import PromptTemplate, HuggingFaceHub, LLMChain, SequentialChain, get_openai_callback



In [56]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [57]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [58]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [59]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json",],
    template=TEMPLATE
)

In [60]:
quiz_chain=LLMChain(llm=llm_hf, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [61]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [62]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)


In [63]:
review_chain=LLMChain(llm=llm_hf, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [64]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [65]:
file_path=r"C:\Users\Administrator\Documents\mcqgen\data.txt"

In [66]:
with open(file_path, 'r') as file:
    TEXT =  file.read()

In [67]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [68]:
NUMBER=5 
SUBJECT="Machine Learning"
TONE="simple"

In [41]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\Administrator\AppData\Local\Temp\ipykernel_31152\2654919708.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of 

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [75]:
response = generate_evaluate_chain.invoke({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON)
})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of 

In [42]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0


In [71]:
quiz=response.get("quiz")

In [76]:
print(response)

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]\n\nFr

In [73]:
quiz

'ML is a field of study in artificial intelligence concerned with the development and study of statistical'

In [72]:
quiz=json.loads(quiz)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [45]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

NameError: name 'quiz' is not defined

In [46]:
quiz=pd.DataFrame(quiz_table_data)

In [47]:
quiz.to_csv("machinelearning.csv", index=False)

In [ ]:
# Define the Hugging Face model (using google/flan-t5-large)
llm = HuggingFaceHub(
    repo_id='google/flan-t5-large',  # Specify the Hugging Face model repo
    model_kwargs={'temperature': 1.5}  # Adjust model parameters like temperature
)

# Define your prompt template for evaluation generation
generate_evaluate_prompt = PromptTemplate(
    template="""Generate an evaluation based on the following:
    Text: {text}
    Number: {number}
    Subject: {subject}
    Tone: {tone}
    Response JSON: {response_json}
    """,
    input_variables=["text", "number", "subject", "tone", "response_json"]
)

# Create the LLMChain using Hugging Face LLM and the prompt template
generate_evaluate_chain = LLMChain(
    llm=llm,
    prompt=generate_evaluate_prompt,
    verbose=True
)

# Sample inputs
TEXT = "This is a sample text."
NUMBER = 5
SUBJECT = "Science"
TONE = "Formal"
RESPONSE_JSON = {"key": "value"}

# Running the LLMChain with inputs
response = generate_evaluate_chain.run({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON)
})

print(response)